# Import libraries

In [2]:
# -- coding:utf-8 --
import pandas as pd
import numpy as np
import os
from os.path import join as pjoin

# from utils import is_number

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

from mpl_toolkits.mplot3d import Axes3D
#import utils
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib as mpl

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Pre Processing

In [4]:
################################
## Read data functions
###############################

def read_train_data(path_train):
    data_df = pd.read_excel(path_train, encoding='gbk', index_col=[0, 1])  # train_sample_375_v2 train_sample_351_v4
    
    # Debugging, SS
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print(data_df)
    data_df = data_df.groupby('PATIENT_ID').last()
    lable = data_df['outcome'].values
    data_df = data_df.drop(['outcome', 'Admission time', 'Discharge time'], axis=1)
    data_df['Type2'] = lable
    data_df = data_df.applymap(lambda x: x.replace('>', '').replace('<', '') if isinstance(x, str) else x)
    data_df = data_df.applymap(lambda x: x if is_number(x) else -1)
    # data_df = data_df.loc[:, data_df.isnull().mean() < 0.2]
    data_df = data_df.astype(float)

    return data_df


def data_preprocess():
    path_train = 'time_series_375_prerpocess_en.xlsx'  # to_ml
    data_df_unna = read_train_data(path_train)

    # Reads the excel as it is, SS
    data_pre_df = pd.read_excel('time_series_test_110_preprocess_en.xlsx', index_col=[0, 1], encoding='gbk')
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print("data_pre_df excel", data_pre_df)

    # Removes NaN, merges rows by filling empty cells form previous rows, SS
    # Sorts them so that the latest row (the one with the most amount of data) becomes the first row, SS
    data_pre_df = merge_data_by_sliding_window(data_pre_df, n_days=1, dropna=True, subset= top3_feats_cols,
                                                     time_form='diff')
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print("data_pre_df sliding window", data_pre_df)

    # Creates a new index and chooses the first row (the row that we need only), SS
    data_pre_df = data_pre_df.groupby('PATIENT_ID').first().reset_index()
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print("data_pre_df sliding group by", data_pre_df)

    # Replaces (<, >) with blank in case that test results came out this way, SS
    data_pre_df = data_pre_df.applymap(lambda x: x.replace('>', '').replace('<', '') if isinstance(x, str) else x)
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print("data_pre_df sliding applymap", data_pre_df)

    # Removes duplicates just in case if there was duplicates, SS
    data_pre_df = data_pre_df.drop_duplicates()
    #with pd.option_context('display.max_rows', 10, 'display.max_columns', 12):  # more options can be specified also
    #  print("data_pre_df sliding drop_dup", data_pre_df)

    return data_df_unna, data_pre_df

    
### is_number in the read data
def is_number(s):
    if s is None:
        s = np.nan

    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    return False

# Testing data